In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [ ]:
import glob
import os
import datetime
from matplotlib import mlab
from pprint import pprint
import pandas as pd
import numpy as np
import re
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

In [ ]:
class LabProcessing():
  """Класс для обработки лабораторных данных"""
  
  def rename_files(self, txt_list):
    """Переименовывает файлы HFP.txt по порядку 1-5"""
    txt_list.sort()
    new_list = []
    i = 1
    for file in txt_list:
      match = re.search(r'\d\d\d\d\D\w\w\w\d', file)[0]
      file_split = file.split(match)
      match2 = re.search(r'\d\d\d\d\D\w\w\w', match)[0]
      new_list.append(file_split[0] + match2 + str(i) + file_split[1])
      i += 1

    l = 0
    while (l < len(txt_list)):
      os.rename(txt_list[l], new_list[l])
      l += 1

  def run_panel(self, path_folder, path_dist, list_of_sens):
      for k in list_of_sens:
          file = k
          open_data = os.path.join(path_folder, 'df_' + file + '.pkl')
          print(open_data)
          df_garm_bd = pd.read_pickle(r'{}'.format(open_data))
          l = {}
          print(len(df_garm_bd))
          for k in range(len(df_garm_bd)):
              if df_garm_bd.iloc[k][0] not in l:
                  l.update({df_garm_bd.iloc[k][0]: df_garm_bd.iloc[k][2]})
              if df_garm_bd.iloc[k][0] in l:
                  s = l[df_garm_bd.iloc[k][0]]
                  s = np.vstack((s, df_garm_bd.iloc[k][2]))
                  l.update({df_garm_bd.iloc[k][0]: s})
        
          for k, v in l.items():
              l.update({k: np.median(v, axis=0)})
          #a = input('Задать глубину построения?(1/0)')
          a = '0'
          if a == '1':
              b = input('Введите интервал(например: 25 30): ')
              bb = [int(k) for k in b.split(' ')]
              l_wit = {}
              for key, value in l.items():
                  if bb[0] < float(key) < bb[1]:
                      l_wit.update({key: value})
              l = l_wit
          else:
              pass

          print('Строиться панель задержек')
          """
          z_arr = []
          for k in l.values():
              z = []
              for kl in k:
                  #print(kl, type(kl))
                  if kl == np.array(0):
                      z_log = 1
                  else:
                      z_log = 20 * np.log10(kl)
                  z.append(z_log)
              z_arr.append(np.array(z))
              """
      
          z = np.array([k for k in l.values()])
          y = (np.array([sorted([round(float(k), 2) for k in list(l.keys())], reverse=True)] * 2047)).transpose()
          x = np.array([[k for k in range(-1023, 1024)]] * len(y))
          print(len(x), len(x[0]), len(y), len(y[0]))
          z = z[:-1, :-1]
          levels = MaxNLocator(nbins=500).tick_values(0, 1)
          cmap = plt.get_cmap('Spectral_r', 100)
          norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

          fig, ax0 = plt.subplots(figsize=(20, 10))
          im = ax0.pcolormesh(x, y, z/z.max(), cmap=cmap, norm=norm)
          fig.colorbar(im, ax=ax0)
          ax0.set_title(f'Панель задержек между дачтиками {file}')
          ax0.set_xlabel('Высота', fontsize = 20)
          ax0.set_ylabel('Задержка', fontsize = 20)
          # cf = ax1.contourf(x[:-1, :-1], y[:-1, :-1], z, levels=levels, cmap=cmap)
          # fig.colorbar(cf, ax=ax1)
          # ax1.set_title('contourf with levels')
          # fig.tight_layout()
          save_fig = os.path.join(path_dist, 'panel' + file + '.png')
          plt.savefig(save_fig)
          plt.close(fig)

  def run_superbase(self, file_list, path_dist):
    print('Создается база. Подождите...')
    max_value = 2**15
    deep_list = []
    time_series_list = []
    for file in file_list:
      with open(file, 'r') as f:
        linel = [line.split(' ') for line in [line.rstrip() for line in f] if len(line) != 0][1:]
        linel_withoutdeep = np.array([k[1:] for k in linel], dtype = float)
        linel_withoutdeep[linel_withoutdeep == 0] = 1 
        linel_withoutdeep = 20*np.nan_to_num(np.log10(linel_withoutdeep/max_value))
        deep_list.append([k[0] for k in linel[1:]])
        time_series_list.append(linel_withoutdeep)

    superbase = []
    deep_list_array = np.array(deep_list, dtype = np.float16)
    i = 0
    for k in deep_list_array[0]:
          superbase.append(
              [k, 
                time_series_list[0][i], 
                time_series_list[1][i], 
                time_series_list[2][i], 
                time_series_list[3][i], 
                time_series_list[4][i], 
                time_series_list[5][i]
              ]
          )
          i += 1
    if not os.path.exists(path_dist):
      os.makedirs(path_dist)
    df_superbase = pd.DataFrame(data=superbase)
    df_superbase.to_pickle(os.path.join(path_dist,'superbase.pkl'))
    print('База данных создана. Путь к файлу: {}'.format(os.path.join(path_dist,'superbase.pkl')))
  

  def xcorr(self, x, y, normed=True, detrend=mlab.detrend_none, maxlags=1023):
    Nx = len(x)
    if Nx != len(y):
        raise ValueError('x and y must be equal length')
    x = detrend(np.asarray(x))
    y = detrend(np.asarray(y))
    correls = np.correlate(x, y, mode = 2)
    if normed: correls /= np.sqrt(np.dot(x, x) * np.dot(y, y))
    if maxlags is None: maxlags = Nx - 1
    if maxlags >= Nx or maxlags < 1: raise ValueError('maxlags must be None or strictly ''positive < %d' % Nx)
    lags = np.arange(-maxlags, maxlags + 1)
    correls = correls[Nx - 1 - maxlags: Nx + maxlags]
    return lags, correls

  def func_cross_time_delay(self, list_of_sens, path_folder, path_dist, maxlags=None):
    df_sp = pd.read_pickle(r'{}'.format(os.path.join(path_folder,'superbase.pkl')))
    print('Считаем кросс-корреляцию')
    print(f'Выбраны следующие пары сенсоров{list_of_sens}')
    for k in list_of_sens:
        s = [int(d) for d in k]
        k = s[0]
        n = s[1]
        level_one={}
        for i in range(len(df_sp)):
            try:
                x = df_sp.iloc[i][k]
                y = df_sp.iloc[i][n]
                if maxlags == None:
                    maxlags = len(x) - 1
                else:
                    maxlags = int(maxlags)
                z, a = self.xcorr(x, y, maxlags=maxlags)
                cv = (np.array([z, a]))
                t = np.argmax((cv[1][:]))
                timedelay = cv[0][t]
                level_one.update({i: [df_sp.iloc[i][0], timedelay,
                                      np.array(a)]})  # убран последний столбец так как малоинформативен
                # level_one.update({i: [df_sp.iloc[i][0], timedelay, np.array(a), np.array(z)]})
            except:
                pass  # есть строки где пустые поля поэтому просто игнорируем их
        df_level_one = pd.DataFrame(level_one)
        df_level_one = df_level_one.transpose()
        df_level_one.to_pickle((os.path.join(path_dist, 'df_' + str(k) + str(n) + '.pkl')))
        print("Файл готов: " + f"{os.path.join(path_dist, 'df_' + str(k) + str(n) + '.pkl')}")

In [ ]:
txt_list = [file for file in glob.glob(os.path.join(path_anul, '*HFP[0-9].txt'))]
txt_list

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

path = path_end = r'path'

file_list = [filename for filename in glob.glob(os.path.join(path, '*.txt'))]
max_value = 2**15
#k = file_list[0] 
filek = open(file_list[0])
elem = len(filek.readlines())
filek.close()
t=0
for k in file_list:
    print(f'Считываем файл целиком {k}')
    with open(k,'r') as file:
        file_read = file.readlines()
    z=0
    slovar_spectr={}
    print('Собираем файл по глубинам, производим STFT Kaiser')
    while z < elem:
        if len(file_read[z])>100:
            mass_one = file_read[z].split(' ')
            mass_one_np = np.array([int(k) for k in mass_one[1:]])
            deep = mass_one[0]
            fft_kaiser= np.fft.rfft(mass_one_np*np.kaiser(len(mass_one_np), 8))
            if deep not in slovar_spectr:
                slovar_spectr.update({deep: fft_kaiser})
            if deep in slovar_spectr:
                b = slovar_spectr[deep]
                b = np.vstack((b, fft_kaiser))
                slovar_spectr.update({deep:b})
        z+=1
    file_read= 0
    
    for k, v in slovar_spectr.items():
        slovar_spectr.update({k: np.median(v, axis = 0)})
    print('Пересчитываем в логарифмический масштаб')
    for k, v in slovar_spectr.items():
        slovar_spectr.update({k: 20*np.log10(v/(2**15))})
    print('Строиться спектрограмма')
    
    z = np.array([abs(k) for k in slovar_spectr.values()])
    y = (np.array([sorted([float(k) for k in list(slovar_spectr.keys())], reverse=True)] * 512)).transpose()
    x = np.array([[k for k in range(0,512)]] * len(y))
    print(len(x), len(x[0]), len(y), len(y[0]), len)
    
    z = z[:-1, :-1]
    levels = MaxNLocator(nbins=500).tick_values(z.min(), z.max())
    cmap = plt.get_cmap('Spectral_r', 100)
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
    
    fig, ax0 = plt.subplots(figsize=(10, 10))
    im = ax0.pcolormesh(x, y, z, cmap=cmap, norm=norm)
    fig.colorbar(im, ax=ax0)
    ax0.set_title('Spectrogram')
    ax0.set_xlabel('Frequency, kHz', fontsize=20)
    ax0.set_ylabel('Deep, m', fontsize=20)
    save_fig = os.path.join(path_end,'spectrogram'+str(t) + '.png')
    print(save_fig)
    plt.savefig(save_fig)
    plt.close()
    x=y=z=v=0
    t+=1